In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
import time
from keras.models import Sequential
from keras.layers import Dense, Activation , BatchNormalization, Dropout
from keras import optimizers
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from joblib import dump, load


#set the path for this code
path=r'xxx'
os.chdir(path)

#Timing
start = time.time()

#preprocessing of raw data
x_min_max_scaler = preprocessing.MinMaxScaler()
y_min_max_scaler = preprocessing.MinMaxScaler()

#save the result
result_dir = 'result'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)
    
#get data from the previous generated data file
train_data = np.load(r'.\train_data\traindata.npy')
vali_data = np.load(r'.\train_data\validata.npy')
test_data = np.load(r'.\train_data\testdata.npy')

#divide x and y
x_train = train_data[:,[0,1,2,3,4,5,6,7,8]]
y_train = train_data[:,[9,10,11,12]]
x_vali = vali_data[:,[0,1,2,3,4,5,6,7,8]]
y_vali = vali_data[:,[9,10,11,12]]
x_test = test_data[:,[0,1,2,3,4,5,6,7,8]]
y_test = test_data[:,[9,10,11,12]]

#standardScaler
x_train_std = x_min_max_scaler.fit_transform(x_train)
y_train_std = y_min_max_scaler.fit_transform(y_train)
x_vali_std = x_min_max_scaler.transform(x_vali)
y_vali_std = y_min_max_scaler.transform(y_vali)
x_test_std = x_min_max_scaler.transform(x_test)
y_test_std = y_min_max_scaler.transform(y_test)

#save the model of preprocessing
clf_x = x_min_max_scaler
clf_y = y_min_max_scaler
dump(clf_x, "stsc/stsc_x.joblib")
dump(clf_y, "stsc/stsc_y.joblib")

#layer number, node number can be changed based on the performance of machine learning model
#batchnormalization and dropout can also be appllied based on the performance
model = Sequential()

model.add(Dense(128,input_shape=(9,)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.2))

model.add(Dense(128))
#model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(Dropout(0.3))

#model.add(Dense(128))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(Dropout(0.3)),

model.add(Dense(4)) 
model.compile(loss='mse',optimizer='adam')

#---------------train data+test data-------------------
#batch size and epochs can be changed based on the performance

hist1=model.fit(x_train_std, y_train_std, batch_size=1024, epochs=3000, verbose=1, validation_data=(x_vali_std,y_vali_std))
loss = model.evaluate(x_test_std, y_test_std, verbose=1)
print(loss)

#possible early stop
#early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=40, verbose=0, mode='auto')
#hist1 = model.fit(x_train_std, y_train_std, epochs=300, batch_size=256, verbose=2,
#                    validation_data=(x_test_std, y_test_std), callbacks=[early_stopping])
#loss = model.evaluate(x_test_std, y_test_std, verbose=0)
#print(loss)


#-------------------save model-------------------
json_string = model.to_json()
open(os.path.join(result_dir, 'model.json'), 'w').write(json_string)
model.save(os.path.join(result_dir, 'model.h5'))

#-------------------save weights----------------------
model.save_weights(os.path.join(result_dir, 'weights.h5'))

#------------------plot-------------------------
plt.plot(hist1.history['loss'], color='b', linewidth=2, label='training')
plt.plot(hist1.history['val_loss'], color='r', linewidth=1, label='validation')

#y_pred=model.predict(x_test_std)
#msepred = mean_squared_error(y_test_std, y_pred)
#print(msepred)
#plt.plot(y_test_std,y_pred)

#print the loss figure
plt.grid()
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.yscale('log')
plt.savefig("loss.png")